In [49]:
import csv, os, random
import pandas as pd
import numpy as np
from pandas import ExcelWriter
from pandas import ExcelFile
from docx import Document
from docx.shared import Inches
from docx.oxml.ns import qn
from docx.shared import Pt
from datetime import datetime

document.styles['Normal'].font.name = u'Microsoft YaHei'
document.styles['Normal'].font.size = Pt(9)
document.styles['Normal']._element.rPr.rFonts.set(qn('w:eastAsia'), u'Microsoft YaHei')

t = datetime.now() 
stamp = f'Issue Date: {t.strftime("%m-%d")}'      
    
def fetch_dataFrame(file_name, n):
    ## read data from excel
    os.chdir(r"/Users/yunjiang/OneDrive/365 Documents")
    df = pd.read_excel(f'{file_name} exam.xlsx', index_col=0).fillna('无')
    lst = []
    while len(lst) <= n-1 :
        i = random.choice(range(1,(len(df.index))))
        if i not in lst:
            lst.append(i)
        else:
            continue
    random_df = pd.DataFrame(df, index = lst).reset_index()
    return random_df

    

def gen_select_answer_data(random_df):
    ## read answer from data
    random_answer = random_df['answer']
    answer_lst = []
    for i in range(len(random_answer)):
        if i%10 == 0 :
            answer_index = f'{i+1}-{i//10*10+10}'
            answer_lst.append(answer_index)
        answer_lst.append(f'{random_answer[i]}')
    return answer_lst
    
def gen_write_answer_data(random_df):
    ## read answer from data
    random_answer = random_df['answer']
    answer_lst = []
    for i in range(len(random_answer)):
        answer_lst.append(f'#{i+1}')
        answer_lst.append(f'{random_answer[i]}')
    return '\n'.join(answer_lst) 


def gen_select_question(random_df, n, one_line_chr_count=55):
    ## modify lines 
    line= ''
    for i in range(n):
        line = line + str(i + 1) + '：' + str(random_df.loc[i][1]) + '\n'
        count = 0
        for n in (2, 3, 4, 5):
            count = count + len(str(random_df.loc[i][n])) + 4
            if count<=one_line_chr_count:
    ## choiceslen跟字体大小调节 小5号字体为60
                line = line + chr(95 + n).upper() + '.' + str(random_df.loc[i][n]) + '   '
            else:
                count = len(str(random_df.loc[i][n]))
                line = line +'\n' + chr(95 + n).upper() + '.' + str(random_df.loc[i][n]) + '   '
        line = line + '\n'
    return line
       

def gen_write_question(random_df,n):
    line= ''
    for i in range(n):
        choiceslen = int(0)
        line = line +  str(i + 1) + '.' + random_df.loc[i][1]
        line = line + '\n'
    return line
        
    
def gen_fleet_questions(fleet, select=20, qrh=2, write=4):
    select_file = f'{fleet} select'
    write_file = f'{fleet} write'
    qrh_file = f'{fleet} qrh'
    
    select_df = fetch_dataFrame(select_file, select)
    select_block = gen_selcet_question(select_df, select)

    qrh_df = fetch_dataFrame(qrh_file, write)
    qrh_block = gen_write_question(qrh_df, write)
    
    write_df = fetch_dataFrame(write_file, write)
    write_block = gen_write_question(write_df, write)
   
    select_answer = gen_select_answer_data(select_df)
    write_answer = gen_write_answer_data(write_df)
    
   
    paragraph = f'''「{fleet}」                                {stamp}  
姓名_____________    员工号________    申请等级___    面试结果____ 成绩______
选择题 每题2分（在左侧题号处写处答案)
{select_block}
熟记项目 不记分，必须全对 (可写在背面)
{qrh_block}
简答题 每题5分 (可写在背面)
{write_block}

------
{select_answer}
{write_answer}
'''
    
    return paragraph


def gen_OM_questions(fleet, select=5, reg=10, write=2):
    select_file = f'{fleet} select'
    write_file = f'{fleet} write'
    reg_file = f'{fleet} reg'
    
    select_df = fetch_dataFrame(select_file, select)
    select_block = gen_select_question(select_df, select)

    reg_df = fetch_dataFrame(reg_file, reg)
    reg_block = gen_write_question(reg_df, reg)
    
    write_df = fetch_dataFrame(write_file, write)
    write_block = gen_write_question(write_df, write)
   
    select_answer = gen_select_answer_data(select_df)
    write_answer = gen_write_answer_data(write_df)
    reg_answer = gen_select_answer_data(reg_df)
   
    paragraph = f'''「运行」                                {stamp}   
姓名_____________    员工号________    申请等级___    面试结果____ 成绩______
选择题 每题2分（在左侧题号处写处答案)
{select_block}
规章 每题2分（在左侧题号处写处答案)
A一般差错/B严重差错/C一般事故征候/D严重事故征候
{reg_block}
简答题 每题5分 (可写在背面)
{write_block}

------
{select_answer}
{reg_answer}
{write_answer}
'''    
    return paragraph


def pub( *fleets):
    for fleet in fleets:
        if fleet == 'OM':
            document = Document()
            p = document.add_paragraph(gen_OM_questions(fleet))
            document.save(f'运行 试卷 {stamp}.docx')
        else:
            document = Document()
            p = document.add_paragraph(gen_fleet_questions(fleet))
            document.save(f'{fleet} 试卷 {stamp}.docx')
    
    

In [50]:
pub('B777', 'B737', 'A320', 'A330', 'A350', 'OM')

In [45]:
print(gen_fleet_questions('B777'))

「B777」                                Issue Date: 2020-02-11  
姓名_____________    员工号________    申请等级___    面试结果____ 成绩______
选择题 每题2分（在左侧题号处写处答案)
1：在OFFPATH DES页面的“TO DRAG”是什么含义？
A.至阻力下降圈的距离   B.到要求拉减速板时机的距离   C.要求减速   D.无   
2：在什么高度襟翼机动速度才能显示？
A.大约10000英尺以下   B.大约15000英尺以下   C.大约20000英尺以下   
D.大约25000英尺以下   
3：右边的EFIS控制面板能控制中央下部的MFD吗？
A.可以   B.不行   C.在特定条件下才可以   D.无   
4：当襟翼5为爬升减推力点时，在起飞前使用襟翼5起飞起飞基准页面“TAKEOFF REF”行输入5时CDU草稿行会显示什么信息？
A.无效输入   B.检查性能   C.没有信息   D.无   
5：单发时TAC工作配平回中电门可以控制方向舵配平吗？
A.可以   B.不行   C.不受影响   D.无   
6：左中右三部VHF哪部仅用于声音通讯
A.左VHF   B.中VHF   C.右VHF   D.中和右VHF   
7：起飞基准页面有效坡度值的起飞性能输入是多少？
A.0.0~2.0   B.0.0~2.5    C.0.0~3.0   D.0.0~4.0   
8：主起落架不一致落地滑梯能在机头抬起时使用吗？
A.能都可以使用   B.不能只能在中后部撤离   C.不需要考虑这一点   D.无   
9：如果发送到ND的资料量超过显示的容量会显示什么信息？
A.显示EXCESS DATA信息   B.显示EXCESS FULL信息   C.显示DATA FULL信息   
D.无   
10：当选择等待页面后在或低于14000英尺时页面的航段时间为多少？
A.1分钟   B.1.5分钟   C.2分钟   D.2.5分钟   
11：当ADIRU没有对准时，V2将显示什么？
A.不显示任何内容   B.显示虚线   C.显示位置是空白   D.无   
12：当ADIRU故障后对FLCH的述说对吗？
A.可以使用FLC

In [43]:
gen_OM_questions('OM')

"「运行」                                Issue Date: 2020-02-11   \n姓名_____________    员工号________    申请等级___    面试结果____ 成绩______\n选择题 每题2分（在左侧题号处写处答案)\n1：地球表面两点之间航程最短的航线是\nA.等角航线   B.圆锥投影图上连接该两点的直线   C.大圆航线   D.圆拄投影图上连接该两点的直线   \n2：当在地面上，QFE值将总是在高度表上显示零高度。\nA.正确   B.错误   C.无   D.无   \n3：起飞后起落架的收上，襟翼的收上，飞机重心________。\nA.向前移动   B.向后移动   C.无   D.无   \n4：两架以上航空器跟进滑行，后航空器不得超越前航空器，后航空器与前航空器的距离，不得小于_____ 米。\nA.30   B.50   C.无   D.无   \n5：正常起飞时，抬机头速率不应大于几度/秒？\nA.2度/秒   B.3度/秒   C.4度/秒   D.1.5度/秒   \n\n规章 每题2分（在左侧题号处写处答案)\nA一般差错/B严重差错/C一般事故征候/D严重事故征候\n1.飞错或擅自改变飞行高度，忘调或错调高度表基准，导致偏离指令高度≥300英尺，˂500英尺\n2.在占用、关闭的跑道上着陆或尝试着陆\n3.雷击、鸟击、雹击造成航空器受损\n4.超过最大重量起飞或着陆，造成飞机受损\n5.调错导航台，造成后果\n6.迷航\n7.飞错航路偏出应飞航迹 ≥5km以上\n8.未按规定操作，航空器受到雷击、鸟击、雹击造成航空器损坏\n9.未按程序飞行，触发襟翼/起落架警告\n10.人为原因导致航空器上仰姿态≥25度，或下俯姿态≥10度，或坡度角≥40度，持续时间≥3秒，出现异常状态\n\n简答题 每题5分 (可写在背面)\n1.滑行过程中如 ATC 部门指令改变起飞方向、起飞跑道或离场程序，应该怎么做？\n2.地面结冰条件是指什么样的天气条件？\n\n\n------\n['1-10', 'C', 'B', 'A', 'B', 'B']\n['1-10', 'A', 'D', 'C', 'C', 'A', 'C', 'B', 'B', 'A', 'B']\n#1\n

In [44]:
print(gen_OM_questions('OM'))

「运行」                                Issue Date: 2020-02-11   
姓名_____________    员工号________    申请等级___    面试结果____ 成绩______
选择题 每题2分（在左侧题号处写处答案)
1：在卫星云图的可见光云图上，大块厚云，积雨云表现为何仲色调
A.白色   B.灰白色   C.灰色   D.无   
2：平均而言，在对流层中每上升100米，大气温度的变化是
A.下降0.65度   B.下降1度   C.上升0.4至0.7度   D.无   
3：飞机在同一高度上交叉相遇，飞行员从座舱左侧看到另一架航空器时应当下降高度，从座舱右侧看到另一架航空器时应当上升高度。
A.正确   B.不正确   C.无   D.无   
4：偏流是哪两条线的夹角？以哪一条线为基准？
A.航迹线和航线，以航线为基准   B.航迹线和航向线，以航迹线为基准   C.航线和航向线，以航向线为基准   
D.航迹线和航向线，以航向线为基准   
5：重心过于靠前或靠后，会有哪些不利的影响________。
A.可能造成飞机在地面产生倾斜   B.使得前轮无载重，造成无法控制滑行或滑跑方向   
C.使得安定面配平处于上偏或下偏的较大位置，造成飞机的气动不稳定   D.以上全部   

规章 每题2分（在左侧题号处写处答案)
A一般差错/B严重差错/C一般事故征候/D严重事故征候
1.机组人员与任务书不符，航空器起飞
2.飞行中出现失速警告，持续时间3秒（含）以上
3.丢失登机牌后未按规定报告
4.着陆过程中飞机冲出跑道
5.在地面与其他航空器或车辆碰撞造成航空器受损或人员轻伤
6.着陆过载≥2.4g
7.无线电高度50英尺至接地时间˂2.5秒，或飞机坡度≥10度（瞬间除外）
8.机组人员与任务书不符，航空器滑出
9.未按规定操作造成航空器损坏或人员受伤
10.认错跑道或跑道方向进近，建立着陆形态

简答题 每题5分 (可写在背面)
1.一般情况下在进行载重平衡计算时，成人旅客、儿童旅客和婴儿分别按多少重量进行计算？
2.出现双向通信失效时，应答机应如何使用？


------
['1-10', 'A', 'A', 'A', 'D', 'D']
['1-10', 'B', 'C', 'A'